# [Module 4.4] API GATEWAY


In [ ]:
def show_top_N_label(score_list, topN):

    import numpy as np

    top_n_idx = np.argsort(score_list)[-topN:]
    top_n_values = [score_list[i] for i in top_n_idx]
    
    print(top_n_idx)


    top_n_idx_list = top_n_idx.tolist()
    top_n_idx_list.reverse()
    top_n_values = [score_list[i] for i in top_n_idx_list]    
    
    print(top_n_idx_list)
    print(top_n_values)

topN = 3    
show_top_N_label(predicted_classes[0], topN)


## Boto3 Client 에서 실행

In [19]:
invoke_endpoint = tweet_bert_endpoint_name

In [20]:
invoke_endpoint

'sagemaker-tensorflow-serving-2020-07-24-10-34-12-001'

In [21]:
%%bash
echo {$invoke_endpoint}

{}


In [25]:
%%bash -s {invoke_endpoint}

invoke_endpoint=$1
echo $invoke_endpoint

aws sagemaker-runtime invoke-endpoint \
  --endpoint-name  $invoke_endpoint \
  --body '["This is great"]' \
  --content-type application/json \
  --accept application/json \
  results
echo $results

sagemaker-tensorflow-serving-2020-07-24-10-34-12-001
{
    "ContentType": "application/json",
    "InvokedProductionVariant": "AllTraffic"
}



In [ ]:
payload = "This is great"
payload = '["' + payload + '"]'
print(payload)

In [ ]:
import json
import os
import io
import boto3
import csv

ENDPOINT_NAME = os.environ['ENDPOINT_NAME'] 
runtime = boto3.client('runtime.sagemaker')


def lambda_handler(event, context):
    # TODO implement
try:
print("Received event: " + json.dumps(event, indent=2))

data = json.loads(json.dumps(event))
payload = data['tweet']
payload = '["' + payload + '"]'
print("Final input: " ,payload)

print(payload)

response = runtime.invoke_endpoint(EndpointName = ENDPOINT_NAME,
                            ContentType = 'application/json',
                            Body = payload)

print(response)
result = json.loads(response['Body'].read().decode())
print(result)
return result
except:
print("An error occurred")
return data


# return {
#     'statusCode': 200,
#     'body': json.dumps('Hello from Lambda!')
# }


## Create Different Inference Call

In [ ]:
import json
import os
import io
import boto3
import csv

ENDPOINT_NAME = os.environ['ENDPOINT_NAME'] 
runtime = boto3.client('runtime.sagemaker')


def lambda_handler(event, context):
    # TODO implement
try:
print("Received event: " + json.dumps(event, indent=2))

data = json.loads(json.dumps(event))
payload = data['tweet']
payload = '["' + payload + '"]'
print("Final input: " ,payload)

print(payload)

response = runtime.invoke_endpoint(EndpointName = ENDPOINT_NAME,
                            ContentType = 'application/json',
                            Body = payload)

print(response)
result = json.loads(response['Body'].read().decode())
print(result)
return result
except:
print("An error occurred")
return data


# return {
#     'statusCode': 200,
#     'body': json.dumps('Hello from Lambda!')
# }
